# Welcome to ProgPy's Horizon Example Notebook! 

When performing a prediction, it is possible to specify the time value to predict to, or the prediction `horizon`. This prediction horizon marks the end of the "time of interest" for the prediction. Often this represents the end of a mission or sufficiently far in the future where the user is unconcerned with the events that occur after this time.

The following example illustrates the use of a `horizon` by performing a prediction with uncertainty given a Prognostics Model with a specific prediction horizon. 


We will use the [ThrownObject](https://nasa.github.io/progpy/api_ref/prog_models/IncludedModels.html#thrownobject) model for this example. Once an instance of this class is created, prediction will occur in two steps: 
1) Generation of samples from the intial state 
2) Prediction of future states (with uncertainty) and the times at which the event thresholds will be reached, within the prediction horizon. All events outside the horizon come back as None and are ignored in metrics. 

The results of this prediction will be:
- Predicted future values (inputs, states, outputs, event_states) with uncertainty from prediction
- Time event is predicted to occur (with uncertainty)

First, let's import the necessary modules.

In [ ]:
import numpy as np
from progpy.models.thrown_object import ThrownObject
from progpy.predictors import MonteCarlo
from progpy.uncertain_data import MultivariateNormalDist
from pprint import pprint

Next, let's define our model. We'll instantiate a `ThrownObject` model with `process_noise` and `measurement_noise` set to 0.25 and 0.2 respectively. Then we'll initialize the model. 

In [ ]:
m = ThrownObject(process_noise = 0.25, measurement_noise = 0.2)
initial_state = m.initialize()

To do prediction, we need a distribution of samples. Here, we'll create a distribution around the initial state using a Multivariate Normal Distribution. We'll use 1000 samples for prediction. 

In [ ]:
NUM_SAMPLES = 1000
x = MultivariateNormalDist(initial_state.keys(), initial_state.values(), np.diag([x_i*0.01 for x_i in initial_state.values()]))

Next, let's set up a predictor. Here, we'll be using the [MonteCarlo](https://nasa.github.io/progpy/prog_algs_guide.html#prog_algs.predictors.MonteCarlo) Predictor.

In [ ]:
mc = predictors.MonteCarlo(m)

Now, let's perform a prediction. We given the `predict` method the following arguments:
- Samples ()
- Number of samples 
- Step size for the prediction 
- 

Finally, the predict function requires a future loading function to be passed. This function is used to generate future inputs to the model. Since our model does not have any inputs, we'll pass a function that returns an empty set.

In [ ]:
PREDICTION_HORIZON = 5
samples = filt.x
STEP_SIZE = 0.001

def future_loading(t, x = None):
    return {}

# Making Prediction
mc_results = mc.predict(samples, future_loading, dt=STEP_SIZE, horizon = PREDICTION_HORIZON)


#### Let's see the results of the predicted time of event!

In [ ]:
metrics = mc_results.time_of_event.metrics()
print("\nPredicted Time of Event:")
print("\nSamples where impact occurs before horizon: {:.2f}%".format(metrics['impact']['number of samples']/100 * 100))
pprint(metrics)  # Note this takes some time
mc_results.time_of_event.plot_hist(keys = 'impact')
mc_results.time_of_event.plot_hist(keys = 'falling')

As you can see from the results...

To conclude...